In [30]:
import numpy as np
import pyvinecopulib as pv
from vcimpute.helper_datagen import make_complete_data_matrix, mask_MCAR
from vcimpute.helper_mdp import sort_miss_vars_by_increasing_miss_vars, mdp_coords, old_to_new
from vcimpute.sakuth import MdpFit
from vcimpute.utils import vfunc

seed = 22
d = 5

X = make_complete_data_matrix(1000, d, 'gaussian', seed=seed)
X_mask = mask_MCAR(X, 'general', 0.2, seed=seed)
del X # tmp

model = MdpFit('gaussian', 10, seed)
X_imp, non_adhoc_patterns = model.fit_transform(X_mask)

sort_miss_vars_by_increasing_miss_vars(non_adhoc_patterns)

[[4],
 [3],
 [1],
 [5, 4],
 [3, 4],
 [2, 4],
 [5, 1],
 [1, 4],
 [3, 1],
 [2, 1],
 [5, 3, 4],
 [2, 5, 4],
 [2, 3, 4],
 [5, 1, 4],
 [3, 1, 4],
 [2, 1, 4]]

In [63]:
miss_vars = np.array([4])
d = 5

miss_idx = miss_vars - 1
mdp = np.zeros(shape=(d,), dtype='bool')
mdp[miss_idx] = True

all_vars = 1 + np.arange(d)
rest_vars = np.setdiff1d(all_vars, miss_vars)
rest_idx = rest_vars -1 

d_sub = len(rest_vars)
old_to_new_map = old_to_new(rest_vars, 1+np.arange(d_sub))
for j,k in zip(miss_vars, range(d_sub,d)):
    old_to_new_map[j]=1+k
new_to_old_map = {v:k for k,v in old_to_new_map.items()}
    

In [32]:
controls = pv.FitControlsVinecop(family_set=[pv.BicopFamily.gaussian], num_threads=10)

cop = pv.Vinecop(d_sub)
cop.select(X_imp[:, rest_idx], controls)

In [33]:
cop

<pyvinecopulib.Vinecop>
** Tree: 0
3,2 <-> Gaussian, parameters = -0.82676
2,1 <-> Gaussian, parameters = -0.17581
1,4 <-> Gaussian, parameters = -0.5761
** Tree: 1
3,1 | 2 <-> Gaussian, parameters = -0.115326
2,4 | 1 <-> Gaussian, parameters = 0.0688451
** Tree: 2
3,4 | 1,2 <-> Gaussian, parameters = 0.047863

In [41]:
# tree0
candidates = rest_vars

In [42]:
idx_mis = 3
var_mis = 4

In [55]:
max_kt = 0
max_kt_idx = None
max_kt_var = None
best_bicop = None
for c in candidates:
    bicop_tmp = pv.Bicop(family=pv.BicopFamily.gaussian)
    vfunc(bicop_tmp.fit, X_imp[:, idx_mis], X_imp[:, c-1])
    curr_kt = bicop_tmp.parameters_to_tau(bicop_tmp.parameters)
    print(c,curr_kt)
    if np.abs(curr_kt) > max_kt:
        max_kt_idx = c-1
        max_kt_var = c
        max_kt = np.abs(curr_kt)
        best_bicop = bicop_tmp

1 0.34472508357796283
2 -0.20297080319803967
3 0.30908397517937
5 -0.12203267006352383


In [ ]:
np.array([
   [1, 2, 1, 4, 4],
   [0, 1, 4, 1, 0],
   [0, 4, 2, 0, 0],
   [0, 3, 0, 0, 0],
   [5, 0, 0, 0, 0]
], dtype=np.uint64)

In [26]:
vfunc(cop0.fit, X_imp[:,3], X_imp[:,4]) # 4,2 copula # fit it and choose maximal
cop0.parameters_to_tau(cop0.parameters)

In [6]:
{2: vfunc(cop.get_pair_copula(0,1).hfunc2, X_imp[:,1], X_imp[:,0]),
 new_to_old_map[4]:vfunc(cop.get_pair_copula(0,1).hfunc1, X_imp[:,0], X_imp[:,new_to_old_map[4]-1])
}

In [28]:
cop0

<pyvinecopulib.Bicop>
Gaussian, parameters = -0.200525

In [4]:

cop0

kendalltau()

cop.get_pair_copula(0,1)

NameError: name 'X_' is not defined

In [ ]:
cop.matrix

In [ ]:
# 1. for many mis var, pick order based on empirical abs kendall tau argmax to existing copula
# add miss var to bottom left
# add rest using h-func
# repeat step 1
# recreate tree and relabel
# impute